In [100]:
import pandas as pd
import datetime as dt
# read the dataset from https://archive.ics.uci.edu/ml/datasets/online+retail
df = pd.read_excel("Online Retail.xlsx")

In [101]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [102]:
# get some statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [103]:
#check for missing values
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [104]:
#drop the missing values
df.dropna(inplace=True)

In [105]:
#create GMV(Gross Merchandise Value from Unit * Price)
df["GMV"] = df["Quantity"]* df["UnitPrice"]
df["GMV"]

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: GMV, Length: 406829, dtype: float64

In [106]:
#  create a date column from timestamp of of invoicing
df['date'] = pd.to_datetime(df['InvoiceDate']).dt.date


In [107]:
#define last day of dataset
lastdate = dt.date(2011,12,9)


# Create And Compute the Recency 

In [108]:
#create recency of each user
recency = df.groupby(by='CustomerID', as_index=False)['date'].max()
recency.columns = ['CustomerID','LastPurschaseDate']
recency

,CustomerID,LastPurschaseDate
0,12346.0,2011-01-18
1,12347.0,2011-12-07
2,12348.0,2011-09-25
3,12349.0,2011-11-21
4,12350.0,2011-02-02
...,...,...
4367,18280.0,2011-03-07
4368,18281.0,2011-06-12
4369,18282.0,2011-12-02
4370,18283.0,2011-12-06


In [109]:
recency['R'] = recency['LastPurschaseDate'].apply(lambda x: (lastdate - x).days)
recency

,CustomerID,LastPurschaseDate,R
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310
...,...,...,...
4367,18280.0,2011-03-07,277
4368,18281.0,2011-06-12,180
4369,18282.0,2011-12-02,7
4370,18283.0,2011-12-06,3


# Create And Compute the Frequency 

In [110]:
df_new = df
# remove duplicates
df_new.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
#calculate frequency of purchases
Frequency = df_new.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
Frequency.columns = ['CustomerID','F']
Frequency

,CustomerID,F
0,12346.0,2
1,12347.0,7
2,12348.0,4
3,12349.0,1
4,12350.0,1
...,...,...
4367,18280.0,1
4368,18281.0,1
4369,18282.0,3
4370,18283.0,16


# Create And Compute the Μonetary 

In [111]:
#create monetary of each customer
monetary = df_new.groupby(by='CustomerID',as_index=False).agg({'GMV': 'sum'})
monetary.columns = ['CustomerID','M']
monetary

,CustomerID,M
0,12346.0,0.00
1,12347.0,163.16
2,12348.0,331.36
3,12349.0,15.00
4,12350.0,25.20
...,...,...
4367,18280.0,23.70
4368,18281.0,5.04
4369,18282.0,36.80
4370,18283.0,66.75


In [112]:
rf = recency.merge(Frequency,on='CustomerID')


In [113]:
rfm = rf.merge(monetary,on='CustomerID')
#use CustomerID as index column
rfm.set_index('CustomerID',inplace=True)
#no longer need it
rfm.drop('LastPurschaseDate',axis=1,inplace=True)
rfm

,R,F,M
CustomerID,,,
12346.0,325,2,0.00
12347.0,2,7,163.16
12348.0,75,4,331.36
12349.0,18,1,15.00
12350.0,310,1,25.20
...,...,...,...
18280.0,277,1,23.70
18281.0,180,1,5.04
18282.0,7,3,36.80


In [114]:
#create quartiles for the recency variable
rfm["R"] = pd.qcut(rfm['R'], 5, labels=[5, 4, 3, 2, 1])


In [115]:
#create quartiles for the frecuency variable

rfm["F"] = pd.qcut(rfm["F"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [116]:
#create quartiles for the monetary variable

rfm["M"] = pd.qcut(rfm['M'], 5, labels=[1, 2, 3, 4, 5])

In [117]:
rfm

,R,F,M
CustomerID,,,
12346.0,1,2,1
12347.0,5,4,5
12348.0,2,3,5
12349.0,4,1,2
12350.0,1,1,2
...,...,...,...
18280.0,1,2,2
18281.0,1,2,1
18282.0,5,3,3


In [118]:
#add them all together r+f+m = RFM
rfm["RFM_Total"] = (rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str))

In [119]:
rfm['RFM_Total']

CustomerID
12346.0    121
12347.0    545
12348.0    235
12349.0    412
12350.0    112
          ... 
18280.0    122
18281.0    121
18282.0    533
18283.0    554
18287.0    334
Name: RFM_Total, Length: 4372, dtype: object

# Customer Segmentation - Bases on RFM Score

In [120]:
#Check for Diffent Segments based on R,F,M Values
print("Best Segment: Top Customers in all categories: ",len(rfm[rfm['RFM_Total']=='555']))
print('Loyal Customers which buy frequently: ',len(rfm[rfm['F']== 5]))
print("High Value Spenders: ",len(rfm[rfm['M']== 5]))
print('Best customers that present churning behavior: ', len(rfm[rfm['RFM_Total']=='255']))
print('Best customers that lost them: ',len(rfm[rfm['RFM_Total']=='155']))
print('Worst Customers that lost them: ',len(rfm[rfm['RFM_Total']=='111']))

Best Segment: Top Customers in all categories:  283
Loyal Customers which buy frequently:  875
High Value Spenders:  875
Best customers that present churning behavior:  25
Best customers that lost them:  5
Worst Customers that lost them:  111


In [121]:
#create a regex map to replace scores with human name segments
segments = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [122]:
#replace bases on values
rfm['segment'] = rfm['RFM_Total'].replace(segments, regex=True)
rfm = rfm[["R", "F", "M", "segment"]]


In [123]:
#keep only characters and remove numbers from regex
rfm['segment'] = rfm['segment'].str.replace('\d+', '')
rfm

<ipython-input-123-a945f6c94ff6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['segment'].str.replace('\d+', '')


,R,F,M,segment
CustomerID,,,,
12346.0,1,2,1,hibernating
12347.0,5,4,5,loyal_customers
12348.0,2,3,5,at_risk
12349.0,4,1,2,hibernating
12350.0,1,1,2,hibernating
...,...,...,...,...
18280.0,1,2,2,hibernating
18281.0,1,2,1,hibernating
18282.0,5,3,3,need_attention


In [124]:
#create a dataset for Tableau Visualization
rfm.to_excel("tableau_rfm_Foteini.xlsx")